# Swaps
- GetSwaps uses musd-pools-mezo


In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests
from mezo.currency_utils import get_token_price, format_musd_currency_columns
from mezo.datetime_utils import format_datetimes
from mezo.data_utils import add_rolling_values, add_pct_change_columns, add_cumulative_columns
from mezo.clients import BigQueryClient, SubgraphClient, Web3Client

load_dotenv(dotenv_path='../.env', override=True)
COINGECKO_KEY = os.getenv('COINGECKO_KEY')

In [33]:
SWAPS_SUBGRAPH = "https://api.goldsky.com/api/public/project_cm6ks2x8um4aj01uj8nwg1f6r/subgraphs/musd-pools-mezo/1.0.0/gn"

SUBGRAPH_HEADERS = {
        "Content-Type": "application/json",
    }

GET_SWAPS = """
query GetSwaps($skip: Int!) {
  swaps(
  first: 1000
  orderBy: timestamp_
  orderDirection: desc
  skip: $skip
  ) {
    timestamp_
    sender
    to
    contractId_
    amount0In
    amount0Out
    amount1In
    amount1Out
    transactionHash_
    block_number
  }
}
"""

In [34]:
# Instantiate subgraph

swaps = SubgraphClient(
    url = SWAPS_SUBGRAPH,
    headers = SUBGRAPH_HEADERS
)

In [35]:
# Fetch subgraph data
swap_data =  swaps.fetch_subgraph_data(GET_SWAPS, 'swaps')

Fetching transactions with skip=0...
Fetching transactions with skip=1000...
Fetching transactions with skip=2000...
Fetching transactions with skip=3000...
Fetching transactions with skip=4000...
Fetching transactions with skip=5000...
Fetching transactions with skip=6000...
Fetching transactions with skip=7000...
No more records found.


In [36]:
# Save vaults data df

swaps_df = pd.DataFrame(swap_data)
display(swaps_df)

,timestamp_,sender,to,contractId_,amount0In,amount0Out,amount1In,amount1Out,transactionHash_,block_number
0,1753122159,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x49294d31704560a45a10c25abae96146d1142af1,0x52e604c44417233b6ccedddc0d640a405caacefb,50000000000000,0,0,5540158354468513482,0x9ed7265761aa8273b6de06efe55c2f0427f91d401b63...,1863018
1,1753122047,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x49294d31704560a45a10c25abae96146d1142af1,0x52e604c44417233b6ccedddc0d640a405caacefb,0,44854176965436,5000000000000000000,0,0x974193c3b688a5c699475f7f7a0abbb661d059c0a773...,1862987
2,1753120287,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x17ae29741714af8c967e6a02e658cf200db8ff6f,0x52e604c44417233b6ccedddc0d640a405caacefb,100000000000000,0,0,11081922992289086168,0x37251480ddde18e7264ed16ef22ec59a1633650036a4...,1862497
3,1753120265,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0xebeaa898439ba87552a2216ded6cf70fee0319a8,0x52e604c44417233b6ccedddc0d640a405caacefb,0,143510194236019,15997845000000000000,0,0x8f938ea0b82259a98330569f1562eb0815f60e8aa9ff...,1862491
4,1753120110,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0xebeaa898439ba87552a2216ded6cf70fee0319a8,0x52e604c44417233b6ccedddc0d640a405caacefb,144375000000000,0,0,15997845581519225847,0x7ae080789705ec502d599648017213afc80de1efe1f7...,1862448
...,...,...,...,...,...,...,...,...,...,...
6371,1752013895,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x52d033e64774f31a8b7562806aa5782cbbd29382,0x52e604c44417233b6ccedddc0d640a405caacefb,250000000000000,0,0,28463880024065974068,0x635f6d5a45a9bee6cdd11b40987141a6ca4225b7ab97...,1555293
6372,1751924558,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0x52e604c44417233b6ccedddc0d640a405caacefb,0,157198903110751,18910000000000000000,0,0x32011a0f2a46f7c3560a3e7c7a341c3ecdad2d784cc8...,1530335
6373,1751924405,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,2010400,0,0,2009989838895249558,0x3afbf42ee8974285284f7bc0a5f621a5022cee2f0a7e...,1530292
6374,1750952336,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0x10906a9e9215939561597b4c8e4b98f93c02031a,0,999896001631977073,1000000,0,0x14869a1d4d2fd0a499d95c70d104132e1d6bc31372c7...,1264305


In [27]:

POOLS_MAP = {
    'musd_btc_pool': '0x52e604c44417233b6CcEDDDc0d640A405Caacefb',
    'musd_musdc_pool': '0xEd812AEc0Fecc8fD882Ac3eccC43f3aA80A6c356',
    'musd_musdt_pool': '0x10906a9E9215939561597b4C8e4b98F93c02031A'
}

In [28]:
swaps_df['pool'] = swaps_df['contractId_'].str.lower().map({k.lower(): v for k, v in POOLS_MAP.items()})

In [29]:
display(swaps_df)

,timestamp_,sender,to,amount0In,amount0Out,amount1In,amount1Out,contractId_,transactionHash_,block_number,pool
0,1753114724,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x28d40df34083ec2e52bc2ea7c23feeae7f7e834c,110000000000000,0,0,12284482354269013010,0x52e604c44417233b6ccedddc0d640a405caacefb,0xaaddfdcc31258d96505c07f6daaa6ae3dccf7c7a39ae...,1860948,NaN
1,1753114663,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x16123522459ff5b8e9d50b0feeb4c838c922cea9,111000000000000,0,0,12402803653901651958,0x52e604c44417233b6ccedddc0d640a405caacefb,0xd29374784e1ecc4816375ef0e03895e18c59ac57852f...,1860931,NaN
2,1753114620,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x50e85bc9a85757e5ee2a1b18d1c33837f2f5164f,122000000000000,0,0,13639615871571969273,0x52e604c44417233b6ccedddc0d640a405caacefb,0x4e990cf5712aeb43762892de6d5fc2e2f0264a600d53...,1860919,NaN
3,1753114344,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x7682dca3ecbde49a1b9afb3e433f1bab3c5886d6,111002220000000,0,0,12417079483058164783,0x52e604c44417233b6ccedddc0d640a405caacefb,0x50bd8ccfec8de915077a022ff8b102b29620fdcc786b...,1860842,NaN
4,1753114275,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0xf91ccda5fbbe6063ce2eec17df4022965a00259c,122000000000000,0,0,13655046627507064575,0x52e604c44417233b6ccedddc0d640a405caacefb,0xf8891e254cc51fbd2d9a9368f8c6ada8c8d507d98efa...,1860823,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6323,1752013895,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x52d033e64774f31a8b7562806aa5782cbbd29382,250000000000000,0,0,28463880024065974068,0x52e604c44417233b6ccedddc0d640a405caacefb,0x635f6d5a45a9bee6cdd11b40987141a6ca4225b7ab97...,1555293,NaN
6324,1751924558,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0,157198903110751,18910000000000000000,0,0x52e604c44417233b6ccedddc0d640a405caacefb,0x32011a0f2a46f7c3560a3e7c7a341c3ecdad2d784cc8...,1530335,NaN
6325,1751924405,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,2010400,0,0,2009989838895249558,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0x3afbf42ee8974285284f7bc0a5f621a5022cee2f0a7e...,1530292,NaN
6326,1750952336,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0,999896001631977073,1000000,0,0x10906a9e9215939561597b4c8e4b98f93c02031a,0x14869a1d4d2fd0a499d95c70d104132e1d6bc31372c7...,1264305,NaN


In [30]:
# Debug the mapping issue
print("Sample contractId_ values:")
print(swaps_df['contractId_'].head())
print("\nUnique contractId_ values:")
print(swaps_df['contractId_'].unique())

print("\nPOOLS_MAP dictionary:")
for k, v in POOLS_MAP.items():
    print(f"{k}: {v}")

# Create lowercase mapping
pools_map_lower = {v.lower(): k for k, v in POOLS_MAP.items()}
print("\nLowercase mapping (address -> pool name):")
for k, v in pools_map_lower.items():
    print(f"{k}: {v}")

print("\nChecking if contractId_ values match any keys:")
for contract_id in swaps_df['contractId_'].unique():
    match = pools_map_lower.get(contract_id.lower())
    print(f"{contract_id} -> {match}")

Sample contractId_ values:
0    0x52e604c44417233b6ccedddc0d640a405caacefb
1    0x52e604c44417233b6ccedddc0d640a405caacefb
2    0x52e604c44417233b6ccedddc0d640a405caacefb
3    0x52e604c44417233b6ccedddc0d640a405caacefb
4    0x52e604c44417233b6ccedddc0d640a405caacefb
Name: contractId_, dtype: object

Unique contractId_ values:
['0x52e604c44417233b6ccedddc0d640a405caacefb'
 '0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356'
 '0x10906a9e9215939561597b4c8e4b98f93c02031a']

POOLS_MAP dictionary:
musd_btc_pool: 0x52e604c44417233b6CcEDDDc0d640A405Caacefb
musd_musdc_pool: 0xEd812AEc0Fecc8fD882Ac3eccC43f3aA80A6c356
musd_musdt_pool: 0x10906a9E9215939561597b4C8e4b98F93c02031A

Lowercase mapping (address -> pool name):
0x52e604c44417233b6ccedddc0d640a405caacefb: musd_btc_pool
0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356: musd_musdc_pool
0x10906a9e9215939561597b4c8e4b98f93c02031a: musd_musdt_pool

Checking if contractId_ values match any keys:
0x52e604c44417233b6ccedddc0d640a405caacefb -> musd_btc_pool
0xe